In [137]:
import psycopg2
import pandas as pd
import numpy as np

In [138]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="postgres"
)

In [139]:
cur = conn.cursor()

In [140]:
cur.execute("""
select 'p' charecter, bank_id, swtc+swtv total, wight
from(
    select bank_id, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
    from(
        select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
        from(
            select *
            from(
                select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                where count = 0
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0

        union all
        select *
        from(
            select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
            from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
            where count != 0) as b1) as b2
    group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
where transaction_type_chain like '%p%'
group by bank_id) as b4
where bank_id = 13335
union all
select 'c' charecter, bank_id, swtc+swtv total, wight
from(
    select bank_id, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
    from(
        select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
        from(
            select *
            from(
                select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                where count = 0
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0
 
        union all
        select *
        from(
            select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
            from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
            where count != 0) as b1) as b2
    group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
where transaction_type_chain like '%c%'
group by bank_id) as b4
where bank_id = 13335
union all
select 's' charecter, bank_id, swtc+swtv total, wight
from(
    select bank_id, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
    from(
        select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
        from(
            select *
            from(
                select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                where count = 0
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0
 
        union all
        select *
        from(
            select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
            from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
            where count != 0) as b1) as b2
    group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
where transaction_type_chain like '%s%'
group by bank_id) as b4
where bank_id = 13335
union all
select 'm' charecter, bank_id, swtc+swtv total, wight
from(
    select bank_id, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
    from(
        select bank_id, transaction_type_chain, sum(new_count) wight, transaction_cost, transaction_value
        from(
            select *
            from(
                select bank_id, transaction_type_chain,  count(count) new_count, transaction_cost, transaction_value
                from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
                where count = 0
                group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b0
 
        union all
        select *
        from(
            select bank_id, transaction_type_chain,  count new_count, transaction_cost, transaction_value
            from (select transaction_type_chain, dest_wallet_id, bank_id, transaction_cost, transaction_value, transaction_gateway_id, count, transaction_type, wallet_nickname, time_internal from data_dataset) as newdataset
            where count != 0) as b1) as b2
    group by bank_id, transaction_type_chain, transaction_cost, transaction_value) as b3
where transaction_type_chain like '%m%'
group by bank_id) as b4
where bank_id = 13335
order by total desc """)

In [141]:
result = cur.fetchall()
pd.DataFrame(result)

,0,1,2,3
0,s,13335,87544747219,1219891
1,p,13335,87532276892,1219889
2,m,13335,87530414114,1140958
3,c,13335,858215940,10397


In [142]:
data = pd.read_csv('tranasaction_dataset.csv', parse_dates=['time_internal'])
data = data.replace(np.nan,0)

In [143]:
tmp_data = data[data['count'] == 0]
tmp_data
dg1 = tmp_data.groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [144]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']]

In [145]:
dg = pd.concat([dg1, dg2])
dg_total = []
dg_total.append(dg[dg['transaction_type_chain'].str.contains('c')])
dg_total.append(dg[dg['transaction_type_chain'].str.contains('m')])
dg_total.append(dg[dg['transaction_type_chain'].str.contains('p')])
dg_total.append(dg[dg['transaction_type_chain'].str.contains('s')])

In [146]:
dg_total2 = dg_total
result = []
for i in range(4):
    result.append([])
    
for i in range(4):
    dg_total2[i] = dg_total[i].groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()
    dg_total2[i] = dg_total2[i].groupby(by=['bank_id']).agg({
        'count' : 'sum',
        'transaction_cost' : 'sum',
        'transaction_value' : 'sum'
    }).reset_index()
    dg_total2[i]['total_value'] = dg_total2[i]['transaction_cost'] + dg_total2[i]['transaction_value']
    dg_total2[i].drop(['transaction_cost', 'transaction_value'], axis='columns', inplace=True)
    your_bank_id = 13335
    result[i] = dg_total2[i][dg_total2[i]['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)
dg = pd.concat([result[0], result[1], result[2], result[3]])
dg.index = ['c', 'm', 'p', 's']
dg = dg[['bank_id', 'total_value', 'count']]
dg.sort_values(['total_value'], ascending=False)

,bank_id,total_value,count
s,13335,87544747219,1219891
p,13335,87532276892,1219889
m,13335,87530414114,1140958
c,13335,858215940,10397
